# RFSoC QPSK Transceiver

> This design is a full QPSK transceiver, which transmits and receives randomly-generated pulse-shaped symbols with full carrier and timing synchronisation. PYNQ is used to visualise the data at both the DAC and ADC side of the RFSoC data converters, as well as visualising various DSP stages throughout the transmit and receive signal path.

 
## Contents    

* [RFSoC QPSK Transceiver](#RFSoC-QPSK-Transceiver)
    * [Import libraries](#Import-libraries)
    * [Download the QPSK bitstream](#Download-the-QPSK-bitstream)
    * [Inspecting the transmit path](#Inspecting-the-transmit-path)
    * [Inspecting the receive path](#Inspecting-the-receive-path)
    * [Reconfigure the RF Data Converters](#Reconfigure-the-RF-Data-Converters)

## Import libraries

Start by including the `xrfdc` drivers so we can configure the RF data converters, `ipywidgets` to make interactive controls, and `rfsoc_qpsk` for the QPSK design.

In [ ]:
import xrfdc
import ipywidgets as ipw

from rfsoc_qpsk import qpsk_overlay, sdr_plots, dma_timer, dict_widget

## Download the QPSK bitstream

In [ ]:
ol = qpsk_overlay.QpskOverlay()

## Inspecting the transmit path

There are a 3 main steps in the QPSK transmit IP signal path:

1. Random symbol generation
2. Pulse shaping
3. Interpolation
  
This design "taps off" this path after the first two stages so we can inspect the signals in Jupyter Lab.
The RF data converter can be reconfigured from Python too - we'll look at that [later](#Reconfigure-the-RF-Data-Converters).

<img src="./img/QPSK_system_block_diagrams_Tx_only.svg" width="700"/>

First we plot our raw QPSK symbols in the time domain.

In [ ]:
plot = sdr_plots.IQTimePlot(
    ol.qpsk_tx.get_many_symbols(N=5),
    500,
    resampling_fun=sdr_plots.resample_pick
)
plot.get_widget()

We can stream new samples into this plot, either using a rolling buffer or keeping all samples. (See `help(plot.add_data)`)

In [ ]:
streamer = dma_timer.DmaTimer(plot.add_data, ol.qpsk_tx.get_symbols, 0.05)
ipw.HBox(streamer.get_widget())

For the pulse shaped signal, look at the frequency domain too. The FFT is accelerated in the PL.

In [ ]:
fs=4000

iq_plot = sdr_plots.IQTimePlot(ol.qpsk_tx.get_many_shaped_time(N=10), fs, w=800)
iq_dt = dma_timer.DmaTimer(iq_plot.add_data, ol.qpsk_tx.get_shaped_time, 0.05)

fa_plot = sdr_plots.HWFreqPlot(ol.qpsk_tx.get_shaped_fft(), fs, avg_n=4, w=800)
fa_dt = dma_timer.DmaTimer(fa_plot.add_frame, ol.qpsk_tx.get_shaped_fft, 0.3)

tab1 = ipw.Tab([ipw.VBox([fa_plot.get_widget(), ipw.HBox(fa_dt.get_widget())]),
                ipw.VBox([iq_plot.get_widget(), ipw.HBox(iq_dt.get_widget())])
               ])
tab1.set_title(0, 'Frequency domain')
tab1.set_title(1, 'Time domain')
tab1

## Inspecting the receive path

The receive side is nearly the inverse of the transmit path (there's just some extra work for properly synchronising).

Again, there are taps off from a few places in the signal path:

1. After decimation
2. After root-raised-cosine filtering
3. After coarse synchronisation
4. and the data output

<img src="./img/QPSK_system_block_diagrams_Rx_only.svg" width="700"/>

Because there are a few different intermediate stages, let's reuse the same cells to plot any of them on-demand.

Define some properties of each tap.

In [ ]:
taps = {
    'Decimated': 
      {'get': ol.qpsk_rx.get_decimated,     'get_many': ol.qpsk_rx.get_many_decimated,     'fs':1024},
    'RRCed': 
      {'get': ol.qpsk_rx.get_rrced,         'get_many': ol.qpsk_rx.get_many_rrced,         'fs':16384},
    'CoarseSynced': 
      {'get': ol.qpsk_rx.get_coarse_synced, 'get_many': ol.qpsk_rx.get_many_coarse_synced, 'fs':4096}
}

Select one of the taps ('Decimated', 'RRCed', or 'CoarseSynced')

In [ ]:
tap = 'RRCed'

In [ ]:
f=taps[tap]['get']
fm=taps[tap]['get_many']
fs=taps[tap]['fs']

f_plot = sdr_plots.IQFreqPlot(fm(), fs)
f_dt = dma_timer.DmaTimer(f_plot.add_frame, fm, 0.3)

iq_plot = sdr_plots.IQTimePlot(fm(), fs, w=800)
iq_dt = dma_timer.DmaTimer(iq_plot.add_data, f, 0.05)

c_plot = sdr_plots.IQConstellationPlot(fm(), plotrange=(0, len(fm())-1), fade=True)
c_dt = dma_timer.DmaTimer(c_plot.add_data, f, 0.05)

tab1 = ipw.Tab([ipw.VBox([iq_plot.get_widget(), ipw.HBox(iq_dt.get_widget())]),
                ipw.VBox([f_plot.get_widget(), ipw.HBox(f_dt.get_widget())]),
                ipw.VBox([c_plot.get_widget(), ipw.HBox(c_dt.get_widget())]),])
tab1.set_title(0, 'Time domain')
tab1.set_title(1, 'Frequency domain')
tab1.set_title(2, 'Constellation')
tab1

And for the final plot, let's look at the synchronised output data. Note that Jupyter Lab can manage multiple windows. Next we're going to play with the RF settings, so you may want to make a new window for this plot by right clicking the plot and selecting `Create New View for Output`.

In [ ]:
d=ol.qpsk_rx.get_many_data()
fs=500

iq_plot = sdr_plots.IQTimePlot(d, fs, w=800)
iq_dg = dma_timer.DmaTimer(iq_plot.add_data, ol.qpsk_rx.get_data, 0.05)

c_plot = sdr_plots.IQConstellationPlot(d, plotrange=(0, len(d)-1), fade=True)
c_dg = dma_timer.DmaTimer(c_plot.add_data, ol.qpsk_rx.get_data, 0.05)

tab1 = ipw.Tab([ipw.VBox([c_plot.get_widget(), ipw.HBox(c_dg.get_widget())]),
                ipw.VBox([iq_plot.get_widget(), ipw.HBox(iq_dg.get_widget())])])
tab1.set_title(0, 'Constellation')
tab1.set_title(1, 'Time domain')
tab1

### Reconfigure the RF Data Converters

So far the RF settings have been controlled by `QpskOverlay` but we can reconfigure these on the fly in python with the `xrfdc` driver.

First of all, consider the DAC block used for the transmit side.

<img src="./img/RF_DAC.svg" width="700"/>

There's a lot of scope for reconfiguration here - see the [IP product guide](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_1/pg269-rf-data-converter.pdf) and `help(ol.dac_block)` for details.

As an example, let's play with the mixer settings. Try changing the mixer frequency below.

In [ ]:
ol.dac_block.MixerSettings = {
    'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
    'EventSource':    xrfdc.EVNT_SRC_IMMEDIATE,
    'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
    'Freq':           84.0, ################### Change me!
    'MixerMode':      xrfdc.MIXER_MODE_C2R,
    'MixerType':      xrfdc.MIXER_TYPE_FINE,
    'PhaseOffset':    0.0
}

The output signal should disappear until the receive side is configured to match the new carrier frequency. Set the new carrier frequency for the ADC (receive) side below.

In [ ]:
ol.adc_block.MixerSettings = {
    'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
    'EventSource':    xrfdc.EVNT_SRC_TILE,
    'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
    'Freq':           84.0, ################### Change me!
    'MixerMode':      xrfdc.MIXER_MODE_R2C,
    'MixerType':      xrfdc.MIXER_TYPE_FINE,
    'PhaseOffset':    0.0
}
ol.adc_block.UpdateEvent(xrfdc.EVENT_MIXER)

Going one step further, `ipywidgets` can be used to make interactive controls for some of these settings.

Define some settings using JSON first.

In [ ]:
import json

schema_json = """
{
  "title": "TX Settings",
  "type": "object",
  "properties": {
    "Freq": {
      "description": "NCO Frequency (MHz)",
      "type": "number",
      "minimum": 0.0,
      "maximum": 100.0,
      "default": 84.0
    },
    "Gain": {
      "description": "Relative Gain",
      "type": "number",
      "minimum": 0.0,
      "maximum": 1.0,
      "default": 1
    }
  }
}
"""

Now define a function that will take a dictionary of these settings and apply them to our system. We must ensure that only one instance of this is run at a given time (using Python's `Lock()`) to prevent Bad Things.

In [ ]:
from threading import Thread, Lock

tx_mutex = Lock()

def update_tx_settings(config):
    # Acquire lock for mutual exclusion on RF driver
    if tx_mutex.acquire(blocking=False):
        if 'Gain' in config:
            ol.qpsk_tx.axi_qpsk_tx.output_gain = int(config['Gain']*(2**32-1))
        if 'Freq' in config:
            mixer_cfg = ol.dac_block.MixerSettings
            mixer_cfg['Freq'] = config['Freq']
            ol.dac_block.MixerSettings = mixer_cfg
        tx_mutex.release()

Generate the ipywidgets and enjoy.

In [ ]:
cfg = dict_widget.DictWidget(json.loads(schema_json))
gui = cfg.interact(callback=update_tx_settings)
display(gui)